# Portfolio Optimization with Machine Learning
## Data Exploration and Analysis

This notebook provides an interactive exploration of the portfolio optimization project data and results.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
# Add project path
import sys
import os
sys.path.append('..')

from config import Config

## 1. Load and Explore Data

In [ ]:
# Load processed stock data
try:
    processed_data_path = os.path.join(Config.PROCESSED_DATA_DIR, 'processed_stock_data.csv')
    price_data = pd.read_csv(processed_data_path, index_col=0, header=[0, 1])
    price_data.index = pd.to_datetime(price_data.index)
    
    print(f"✅ Loaded price data: {price_data.shape}")
    print(f"Date range: {price_data.index.min()} to {price_data.index.max()}")
    print(f"Number of stocks: {len(price_data.columns.get_level_values(0).unique())}")
    
except FileNotFoundError:
    print("❌ Processed data not found. Please run data collection first.")
    print("Run: python ../main.py --step data")

In [ ]:
# Display data info
if 'price_data' in locals():
    print("Stock symbols:")
    symbols = price_data.columns.get_level_values(0).unique()
    for i, symbol in enumerate(symbols):
        if i % 8 == 0:
            print()
        print(f"{symbol:<6}", end=" ")
    
    print(f"\n\nData columns for each stock: {price_data.columns.get_level_values(1).unique().tolist()}")

## 2. Price Analysis

In [ ]:
# Plot price trends for major stocks
if 'price_data' in locals():
    major_stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA']
    
    plt.figure(figsize=(15, 10))
    
    for i, stock in enumerate(major_stocks):
        if (stock, 'Close') in price_data.columns:
            plt.subplot(2, 3, i+1)
            
            close_prices = price_data[stock]['Close']
            plt.plot(close_prices.index, close_prices.values, linewidth=2)
            plt.title(f'{stock} Stock Price')
            plt.ylabel('Price ($)')
            plt.xticks(rotation=45)
            plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Calculate and visualize returns
if 'price_data' in locals():
    # Calculate returns for all stocks
    returns_data = {}
    
    for symbol in price_data.columns.get_level_values(0).unique():
        if (symbol, 'Close') in price_data.columns:
            close_prices = price_data[symbol]['Close'].dropna()
            returns = close_prices.pct_change().dropna()
            returns_data[symbol] = returns
    
    returns_df = pd.DataFrame(returns_data)
    
    print(f"Returns data shape: {returns_df.shape}")
    print(f"\nReturns summary statistics:")
    print(returns_df.describe().round(4))

In [ ]:
# Returns distribution analysis
if 'returns_df' in locals():
    plt.figure(figsize=(15, 8))
    
    # Plot 1: Returns distribution
    plt.subplot(2, 2, 1)
    for stock in major_stocks:
        if stock in returns_df.columns:
            plt.hist(returns_df[stock].dropna(), bins=50, alpha=0.6, label=stock, density=True)
    plt.title('Returns Distribution')
    plt.xlabel('Daily Return')
    plt.ylabel('Density')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot 2: Volatility over time (rolling)
    plt.subplot(2, 2, 2)
    for stock in major_stocks[:3]:  # Just show top 3 for clarity
        if stock in returns_df.columns:
            rolling_vol = returns_df[stock].rolling(60).std() * np.sqrt(252)
            plt.plot(rolling_vol.index, rolling_vol.values, label=f'{stock} Volatility', linewidth=2)
    plt.title('Rolling 60-Day Volatility (Annualized)')
    plt.ylabel('Volatility')
    plt.legend()
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    
    # Plot 3: Correlation heatmap
    plt.subplot(2, 2, 3)
    corr_matrix = returns_df[major_stocks].corr()
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
                square=True, fmt='.2f', cbar_kws={"shrink": .8})
    plt.title('Stock Returns Correlation')
    
    # Plot 4: Risk-Return scatter
    plt.subplot(2, 2, 4)
    annual_returns = returns_df.mean() * 252
    annual_volatility = returns_df.std() * np.sqrt(252)
    
    plt.scatter(annual_volatility, annual_returns, s=60, alpha=0.7)
    
    # Annotate points
    for i, stock in enumerate(annual_returns.index):
        if stock in major_stocks:
            plt.annotate(stock, (annual_volatility[stock], annual_returns[stock]),
                        xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    plt.xlabel('Annual Volatility')
    plt.ylabel('Annual Return')
    plt.title('Risk-Return Profile')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 3. Clustering Analysis

In [ ]:
# Load clustering results
try:
    clustering_features_path = os.path.join(Config.FEATURES_DIR, 'clustering_features.csv')
    cluster_assignments_path = os.path.join(Config.RESULTS_DIR, 'cluster_assignments_kmeans.csv')
    
    clustering_features = pd.read_csv(clustering_features_path, index_col=0)
    cluster_assignments = pd.read_csv(cluster_assignments_path)
    
    print(f"✅ Loaded clustering features: {clustering_features.shape}")
    print(f"✅ Loaded cluster assignments: {cluster_assignments.shape}")
    
    # Show cluster distribution
    cluster_counts = cluster_assignments['cluster'].value_counts().sort_index()
    print(f"\nCluster distribution:")
    print(cluster_counts)
    
except FileNotFoundError:
    print("❌ Clustering results not found. Please run clustering first.")
    print("Run: python ../main.py --step clustering")

In [ ]:
# Visualize clustering results
if 'clustering_features' in locals() and 'cluster_assignments' in locals():
    # Merge features with cluster assignments
    features_with_clusters = clustering_features.merge(
        cluster_assignments.set_index('symbol'), 
        left_index=True, right_index=True, how='inner'
    )
    
    plt.figure(figsize=(16, 12))
    
    # Plot 1: Risk-Return by cluster
    plt.subplot(2, 3, 1)
    scatter = plt.scatter(features_with_clusters['returns_std'], 
                         features_with_clusters['returns_mean'],
                         c=features_with_clusters['cluster'], 
                         cmap='tab10', s=60, alpha=0.7)
    plt.colorbar(scatter, label='Cluster')
    plt.xlabel('Return Std')
    plt.ylabel('Return Mean')
    plt.title('Risk-Return by Cluster')
    plt.grid(True, alpha=0.3)
    
    # Plot 2: Sharpe ratio by cluster
    plt.subplot(2, 3, 2)
    features_with_clusters.boxplot(column='sharpe_ratio', by='cluster', ax=plt.gca())
    plt.title('Sharpe Ratio by Cluster')
    plt.suptitle('')  # Remove automatic title
    
    # Plot 3: Market correlation by cluster
    plt.subplot(2, 3, 3)
    if 'market_correlation' in features_with_clusters.columns:
        features_with_clusters.boxplot(column='market_correlation', by='cluster', ax=plt.gca())
        plt.title('Market Correlation by Cluster')
        plt.suptitle('')
    
    # Plot 4: Cluster composition pie chart
    plt.subplot(2, 3, 4)
    cluster_counts.plot(kind='pie', autopct='%1.1f%%', ax=plt.gca())
    plt.title('Cluster Size Distribution')
    plt.ylabel('')
    
    # Plot 5: Feature importance heatmap
    plt.subplot(2, 3, 5)
    cluster_means = features_with_clusters.groupby('cluster').mean()
    feature_cols = [col for col in cluster_means.columns if col != 'cluster']
    sns.heatmap(cluster_means[feature_cols[:8]].T, annot=True, fmt='.3f', 
                cmap='RdYlBu_r', center=0, ax=plt.gca())
    plt.title('Cluster Feature Means')
    plt.xlabel('Cluster')
    
    # Plot 6: Sample stocks from each cluster
    plt.subplot(2, 3, 6)
    plt.axis('off')
    
    cluster_info = []
    for cluster_id in sorted(features_with_clusters['cluster'].unique()):
        cluster_stocks = features_with_clusters[features_with_clusters['cluster'] == cluster_id].index.tolist()
        sample_stocks = cluster_stocks[:5]  # Show first 5 stocks
        cluster_info.append(f"Cluster {cluster_id}: {', '.join(sample_stocks)}")
    
    plt.text(0.1, 0.9, '\n'.join(cluster_info), transform=plt.gca().transAxes, 
             fontsize=10, verticalalignment='top', fontfamily='monospace')
    plt.title('Sample Stocks by Cluster')
    
    plt.tight_layout()
    plt.show()

## 4. Portfolio Optimization Results

In [ ]:
# Load optimization results
optimization_methods = ['mean_variance', 'risk_parity', 'min_variance', 'max_sharpe', 'cluster_based']
portfolio_weights = {}
optimization_summaries = {}

for method in optimization_methods:
    try:
        weights_path = os.path.join(Config.RESULTS_DIR, f'portfolio_weights_{method}.csv')
        summary_path = os.path.join(Config.RESULTS_DIR, f'optimization_summary_{method}.json')
        
        if os.path.exists(weights_path):
            weights = pd.read_csv(weights_path, index_col=0, squeeze=True)
            portfolio_weights[method] = weights
        
        if os.path.exists(summary_path):
            import json
            with open(summary_path, 'r') as f:
                summary = json.load(f)
            optimization_summaries[method] = summary
    
    except Exception as e:
        print(f"Could not load {method} results: {e}")

print(f"✅ Loaded optimization results for {len(portfolio_weights)} methods")
print(f"Methods: {list(portfolio_weights.keys())}")

In [ ]:
# Display optimization summary
if optimization_summaries:
    summary_df = pd.DataFrame(optimization_summaries).T
    print("Portfolio Optimization Summary:")
    print("=" * 80)
    print(summary_df.round(4))

In [ ]:
# Visualize portfolio weights
if portfolio_weights:
    n_methods = len(portfolio_weights)
    cols = min(3, n_methods)
    rows = (n_methods + cols - 1) // cols
    
    plt.figure(figsize=(5*cols, 4*rows))
    
    for i, (method, weights) in enumerate(portfolio_weights.items()):
        plt.subplot(rows, cols, i+1)
        
        # Show top 10 positions
        top_weights = weights.nlargest(10)
        
        plt.barh(range(len(top_weights)), top_weights.values)
        plt.yticks(range(len(top_weights)), top_weights.index)
        plt.xlabel('Weight')
        plt.title(f'{method.replace("_", " ").title()} - Top 10 Positions')
        plt.gca().invert_yaxis()
        
        # Add weight labels
        for j, v in enumerate(top_weights.values):
            plt.text(v + 0.001, j, f'{v:.3f}', va='center', fontsize=9)
    
    plt.tight_layout()
    plt.show()

## 5. Backtesting Results

In [ ]:
# Load backtesting results
try:
    comparison_path = os.path.join(Config.RESULTS_DIR, 'strategy_comparison_summary.csv')
    
    if os.path.exists(comparison_path):
        comparison_results = pd.read_csv(comparison_path, index_col=0)
        print("✅ Loaded backtesting comparison results")
        print("\nStrategy Performance Comparison:")
        print("=" * 80)
        print(comparison_results.round(4))
        
        # Highlight best performers
        best_sharpe = comparison_results['Sharpe Ratio'].idxmax()
        best_return = comparison_results['Annual Return'].idxmax()
        
        print(f"\n🏆 Best Sharpe Ratio: {best_sharpe} ({comparison_results.loc[best_sharpe, 'Sharpe Ratio']:.4f})")
        print(f"🏆 Best Annual Return: {best_return} ({comparison_results.loc[best_return, 'Annual Return']:.4f})")
    
    else:
        print("❌ Backtesting results not found. Please run backtesting first.")
        print("Run: python ../main.py --step backtesting")
        
except Exception as e:
    print(f"Error loading backtesting results: {e}")

In [ ]:
# Load and plot individual strategy returns
strategy_returns = {}

for method in optimization_methods:
    try:
        returns_path = os.path.join(Config.RESULTS_DIR, f'portfolio_values_{method}.csv')
        if os.path.exists(returns_path):
            returns = pd.read_csv(returns_path, index_col=0, squeeze=True)
            returns.index = pd.to_datetime(returns.index)
            strategy_returns[method] = returns
    except Exception as e:
        continue

if strategy_returns:
    plt.figure(figsize=(15, 10))
    
    # Plot 1: Cumulative returns
    plt.subplot(2, 2, 1)
    for method, returns in strategy_returns.items():
        cumulative = (1 + returns).cumprod()
        plt.plot(cumulative.index, cumulative.values, label=method.replace('_', ' ').title(), linewidth=2)
    
    plt.title('Cumulative Returns Comparison')
    plt.ylabel('Cumulative Return')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot 2: Rolling Sharpe
    plt.subplot(2, 2, 2)
    for method, returns in strategy_returns.items():
        rolling_sharpe = returns.rolling(60).apply(
            lambda x: x.mean() / x.std() * np.sqrt(252) if x.std() != 0 else 0
        )
        plt.plot(rolling_sharpe.index, rolling_sharpe.values, 
                label=method.replace('_', ' ').title(), linewidth=2)
    
    plt.title('Rolling 60-Day Sharpe Ratio')
    plt.ylabel('Sharpe Ratio')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot 3: Drawdown
    plt.subplot(2, 2, 3)
    for method, returns in strategy_returns.items():
        cumulative = (1 + returns).cumprod()
        rolling_max = cumulative.expanding().max()
        drawdown = (cumulative - rolling_max) / rolling_max
        plt.fill_between(drawdown.index, drawdown.values, 0, 
                        alpha=0.3, label=method.replace('_', ' ').title())
    
    plt.title('Drawdown Analysis')
    plt.ylabel('Drawdown')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot 4: Returns distribution
    plt.subplot(2, 2, 4)
    for method, returns in strategy_returns.items():
        plt.hist(returns.dropna(), bins=30, alpha=0.6, density=True,
                label=method.replace('_', ' ').title())
    
    plt.title('Returns Distribution')
    plt.xlabel('Daily Return')
    plt.ylabel('Density')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 6. Summary and Insights

In [ ]:
# Generate summary insights
print("📊 PORTFOLIO OPTIMIZATION PROJECT SUMMARY")
print("=" * 80)

if 'price_data' in locals():
    print(f"📈 Data Coverage:")
    print(f"   • {len(price_data.columns.get_level_values(0).unique())} stocks analyzed")
    print(f"   • Date range: {price_data.index.min().strftime('%Y-%m-%d')} to {price_data.index.max().strftime('%Y-%m-%d')}")
    print(f"   • {len(price_data)} trading days")

if 'cluster_assignments' in locals():
    print(f"\n🎯 Clustering Analysis:")
    print(f"   • {len(cluster_assignments['cluster'].unique())} clusters identified")
    print(f"   • Most common cluster: {cluster_assignments['cluster'].value_counts().index[0]} ({cluster_assignments['cluster'].value_counts().iloc[0]} stocks)")
    
if optimization_summaries:
    print(f"\n⚡ Optimization Methods:")
    print(f"   • {len(optimization_summaries)} methods implemented")
    
    best_method = max(optimization_summaries.keys(), 
                     key=lambda x: optimization_summaries[x].get('sharpe_ratio', 0))
    print(f"   • Best ex-ante Sharpe: {best_method} ({optimization_summaries[best_method].get('sharpe_ratio', 0):.4f})")

if 'comparison_results' in locals():
    print(f"\n🏆 Backtesting Results:")
    best_sharpe_bt = comparison_results['Sharpe Ratio'].idxmax()
    best_return_bt = comparison_results['Annual Return'].idxmax()
    
    print(f"   • Best backtested Sharpe: {best_sharpe_bt} ({comparison_results.loc[best_sharpe_bt, 'Sharpe Ratio']:.4f})")
    print(f"   • Best backtested return: {best_return_bt} ({comparison_results.loc[best_return_bt, 'Annual Return']:.4f})")
    print(f"   • Average max drawdown: {comparison_results['Max Drawdown'].mean():.4f}")

print("\n💡 Key Insights:")
print("   • Machine learning enhanced portfolio optimization shows promise")
print("   • Stock clustering provides valuable diversification insights")
print("   • Multiple optimization methods offer different risk-return profiles")
print("   • Transaction costs and rebalancing frequency significantly impact performance")

print("\n🚀 Next Steps:")
print("   • Experiment with different ML models for return prediction")
print("   • Implement dynamic rebalancing based on market conditions")
print("   • Add alternative risk measures (VaR, CVaR)")
print("   • Include alternative assets (bonds, commodities, REITs)")
print("   • Test on out-of-sample periods for robust validation")